# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sponge_bob_magic.datasets.movielens import MovieLens
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic.metrics import metrics

from sponge_bob_magic.models.popular_recomennder import PopularRecommender
from sponge_bob_magic.models.als_recommender import ALSRecommender
from sponge_bob_magic.models.knn_recommender import KNNRecommender
from sponge_bob_magic.models.lightfm_recommender import LightFMRecommender

from sponge_bob_magic.scenarios.main_scenario.main_factory import MainScenarioFactory
from sponge_bob_magic.utils import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [3]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [4]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [5]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings

,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116


users

,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067


items

,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [6]:
popular_recommender = PopularRecommender()
als_recommender = ALSRecommender()
knn_recommender = KNNRecommender()
lightfm_recommender = LightFMRecommender()

In [7]:
log_bydate_splitter = log_splitter.LogSplitByDateSplitter(
    True, True,
    datetime(2007, 1, 1)
)
log_random_splitter = log_splitter.LogSplitRandomlySplitter(
    True, True,
    test_size=0.3, seed=1234
)
log_cold_splitter = log_splitter.ColdUsersExtractingSplitter(
    True, True,
    test_size=0.3
)
user_random_splitter = user_log_splitter.RandomUserLogSplitter(
    True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)
user_bydate_splitter = user_log_splitter.ByTimeUserLogSplitter(
    True, True,
    item_test_size=0.3,
    seed=1234,
    user_test_size=500
)

In [8]:
factory = MainScenarioFactory()

### Обучение сценария <a name="fit-scenario"></a>

In [9]:
results = None

In [11]:
scenario = factory.get(
    splitter=user_random_splitter,
    recommender=lightfm_recommender,
    criterion=metrics.HitRateMetric(),
    metrics=[
        metrics.NDCGMetric(),
        metrics.PrecisionMetric(),
        metrics.MAPMetric(),
        metrics.RecallMetric(),
        metrics.Surprisal(log),
    ],
    fallback_recommender=popular_recommender,
)

In [12]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [13]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

30-Jan-20 15:14:13, root, DEBUG: Деление лога на обучающую и тестовую выборку


30-Jan-20 15:14:33, root, DEBUG: Длина трейна и теста: (84013, 15944)


30-Jan-20 15:14:34, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490


30-Jan-20 15:14:35, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355


30-Jan-20 15:14:35, root, DEBUG: Обучение и предсказание дополнительной модели


...

30-Jan-20 15:16:22, root, DEBUG: -- Метрики: HitRate@K=0.8571 nDCG@k=0.2393 Precision@k=0.2298 MAP@k=0.0422 Recall@K=0.1006 Surprisal@K=1.6841


[I 2020-01-30 15:16:22,872]

 Finished trial#1 resulted in value: 0.8571428571428571. Current best value is 0.863265306122449 with parameters: {'rank': 65}.

30-Jan-20 15:16:22, root, DEBUG: Лучшие значения метрики: 0.863265306122449


30-Jan-20 15:16:22, root, DEBUG: Лучшие параметры: {'rank': 65}


In [14]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

,number,value,datetime_start,datetime_complete,params_rank,user_attrs_MAP@k,user_attrs_Precision@k,user_attrs_Recall@K,user_attrs_Surprisal@K,user_attrs_nDCG@k,system_attrs__number,state
0,0,0.863265,2020-01-30 15:14:45.681301,2020-01-30 15:15:35.035806,65,0.043421,0.232449,0.101361,1.671360,0.243132,0,COMPLETE
1,1,0.857143,2020-01-30 15:15:35.128251,2020-01-30 15:16:22.872612,59,0.042154,0.229796,0.100556,1.684121,0.239260,1,COMPLETE


### Получение рекомендаций <a name="predict-scenario"></a>

In [15]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

30-Jan-20 15:16:22, root, DEBUG: Проверка датафреймов


30-Jan-20 15:16:23, root, DEBUG: Предварительная стадия обучения (pre-fit)


30-Jan-20 15:16:23, root, DEBUG: Основная стадия обучения (fit)


30-Jan-20 15:16:23, root, DEBUG: Построение модели LightFM


30-Jan-20 15:16:25, root, DEBUG: Проверка датафреймов


30-Jan-20 15:16:26, root, DEBUG: Выделение дефолтных юзеров


30-Jan-20 15:16:26, root, DEBUG: Выделение дефолтных айтемов


In [16]:
recs.show()

+-------+-------+-------------------+----------+
|user_id|item_id|          relevance|   context|
+-------+-------+-------------------+----------+
|     91|    510| 1.0401673316955566|no_context|
|     91|    511| 0.8517758250236511|no_context|
|     91|    187| 0.8419424891471863|no_context|
|     91|    199| 0.8282949328422546|no_context|
|     91|    300| 0.7915161848068237|no_context|
|     91|    205| 0.7819991707801819|no_context|
|     91|    661| 0.4486207067966461|no_context|
|     91|    197|0.34345322847366333|no_context|
|     91|    480|0.22408486902713776|no_context|
|     91|    172| 0.1867961883544922|no_context|
|    152|     88| 1.1491700410842896|no_context|
|    152|     15|  1.075821042060852|no_context|
|    152|    393| 0.9786251783370972|no_context|
|    152|    237| 0.6142179369926453|no_context|
|    152|    111| 0.6070161461830139|no_context|
|    152|     66| 0.5071985721588135|no_context|
|    152|    402|0.30981165170669556|no_context|
|    152|    451| 0.